In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#import necessary libraries
from pandas import DataFrame
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, scale
import seaborn as sns



pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline

In [3]:
#Load the dataset

cars = pd.read_csv("../input/car-sales/cars_cleaned_by_helen.csv", low_memory=False )
cars.shape

In [4]:
#identify outliners 
plt.figure(figsize = (15,12)) 

sns.scatterplot(x=cars['body_type'], y =cars['mileage'], data = cars)

In [5]:
car=cars[cars['mileage']<=500000]
car.shape

In [6]:
car.head(5)

In [7]:
car.info()

In [8]:
#delete the first column without name 
del car[car.columns[0]]

In [9]:
#drop some necessary features, as power value = horsepower, % of has_accident = % of frame_damaged = % of theft_title =% of salvage , engine_shape is similar to engine_type_simple,
#dont understand the definitely of trime name,listing_id has no meaning, franchise_make is similar to make_name, keep franchise_dealer to identify franchise 
car= car.drop( ['dealer_zip','torque_2','power_1','power_2','latitude','frame_damaged','salvage', 'theft_title','engine_shape','franchise_make','trim_name','major_options','longitude', 'listing_id','vin','listed_date' ], axis = 1)
car.shape

In [10]:
car.info()

In [11]:
#change data type 
car['has_accidents']= car['has_accidents'].astype('bool')


In [12]:
# Decide which categorical variables you want to use in model
for col_name in car.columns:
    if car[col_name].dtypes == 'object':
        unique_cat = len(car[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique categories".format(col_name=col_name, unique_cat=unique_cat))


In [13]:
#Find out the numerical features for feature selection
Num_type= ['int64', 'float64']
numerical_col = list(car.select_dtypes(include = Num_type).columns) 
numerical_col

In [14]:

clean_car_numeric = car[numerical_col]
clean_car_numeric.head(3)

In [15]:
clean_car_numeric.shape

In [16]:
clean_car_numeric.boxplot(figsize = (20,10))
plt.show()

In [17]:
clean_car_numeric['height'] = scale(clean_car_numeric['height'])
clean_car_numeric['length'] = scale(clean_car_numeric['length'])
clean_car_numeric['width'] = scale(clean_car_numeric['width'])

In [18]:
'''
def pearson_corr()
    pearson_corr = clean_car_numeric.corr(method= 'pearson') 

    return pearson_corr

df = pearson_corr()
'''
clean_car_numeric.corr(method= 'pearson') 

In [19]:
plt.figure(figsize = (13,10)) 

sns.heatmap(clean_car_numeric.corr(method= 'pearson'), linewidth = 1,vmin= 0, vmax =1,cmap="YlGnBu",  annot= True, annot_kws= {'size', 10} ) 

plt.title ('Pearson Correlations', fantasize = 25) 

plt.show() 

In [ ]:
plt.figure(figsize = (13,10)) 

#mask
mask= np.triu(np.ones_like(pearson_corr, dtype = np.bool))

#adjust mask and dataset 
mask = mask[1:, :-1]
pearson_corr = pearson_corr.iloc[1:,:-1].copy()

#color map 
camp = sns.diverging_palette(0,230, 90, 60, as_cmap=True )

#plot heatmap 

sns.heatmap(pearson_corr, mask = mask, linewidth = 5,vmin= 0, vmax =1,cmap=camp, 
            fmt=".2f",  cbar_kws= {"shrink":0.8}, square = True ) 

#ticks 
yticks = [i.upper() for i in pearson_corr.index]
xticks = [i.upper() for i in pearson_corr.columns]

plt.yticks(plt.yticks()[0], labels = yticks, rotation = 0)
plt.xticks(plt.xticks()[0], labels = xticks)

plt.title ('Pearson Correlations', fantasize = 25) 

plt.show() 

In [ ]:
#create scatter plot to understand this attribute

plt.figure(figsize = (20,10)) 
sns.pairplot(clean_car_numeric, kind= 'scatter')

plt.show()

In [ ]:
#Spearman Correlation
spearman_corr = clean_car_numeric.corr(method= 'spearman') 

spearman_corr

In [ ]:
plt.figure(figsize = (13,10)) 

sns.heatmap(spearman_corr, linewidth = 1,vmin= 0, vmax =1,cmap="YlGnBu",  annot= True, annot_kws= {'size', 10} ) 

plt.title ('Spearman Correlations', fantasize = 25) 

plt.show() 

In [ ]:
# Kendall Correlation

Kendall_corr = clean_car_numeric.corr(method= 'kendall') 

Kendall_corr

In [ ]:
plt.figure(figsize = (13,10)) 

sns.heatmap(Kendall_corr, linewidth = 1,vmin= 0, vmax =1,cmap="YlGnBu",  annot= True, annot_kws= {'size', 10} ) 

plt.title ('Kendall Correlations', fantasize = 25) 

plt.show() 

SelectKBest allows to specify the statistical technique that you want to use to select 
the k most relevant feature.the techique we use is f_regression. it is univariate linear 
regression test , which calcualte the correlation between each regressor and the target 

In [ ]:
from sklearn.feature_selection import SelectKBest 
from sklearn.feature_selection import f_regression



In [ ]:
# Perform feature selection 
target = clean_car_numeric['price']
features = clean_car_numeric.drop('price',axis = 1)

In [ ]:
select_univeriate = SelectKBest(f_regression, k= 5).fit(features, target)

In [ ]:
feature_mask = select_univariate.get_support()
feature_mask

In [ ]:
features.columns[feature_mask]

In [ ]:
select_univeriate.scores_

In [ ]:
#display what feature was associate with what score clearly 

pd.DataFrame({'FeatureName':features.columns, 'Score':select_univariate.scores_}).sort_values(by='Score',ascendig = False)

In [ ]:
# select the important features 
uni_df = pd.DataFrame({'Univariate Method': features.columns[feature_mask]})
uni_df 

In [ ]:
!pip install mixtend  --upgrade